#### Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
import string

#### Import the datasets

In [2]:
fake = pd.read_csv("./data/Fake.csv")
true = pd.read_csv("./data/True.csv")

In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


#### Create class labels

In [5]:
fake["class"] = 0
true["class"] = 1

In [6]:
fake.shape, true.shape

((23481, 5), (21417, 5))

In [7]:
fake_mannual_test = fake.tail(10)
for i in range(23480, 23470, -1):
    fake.drop([i], axis = 0, inplace=True)

true_mannual_test = true.tail(10)
for i in range(21416, 21406, -1):
    true.drop([i], axis = 0, inplace=True)

In [8]:
fake.shape, true.shape

((23471, 5), (21407, 5))

In [9]:
fake_mannual_test['class'] = 0
true_mannual_test['class'] = 1

C:\Users\aarad\AppData\Local\Temp\ipykernel_36664\3188397402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_mannual_test['class'] = 0
C:\Users\aarad\AppData\Local\Temp\ipykernel_36664\3188397402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_mannual_test['class'] = 1


#### Merge the datasets

In [10]:
data_merged = pd.concat([fake, true], axis = 0)
data_merged.shape

(44878, 5)

In [11]:
data_merged.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [12]:
data = data_merged.drop(['title', 'subject', 'date'], axis = 1)

#### Check for any Null Values and Duplicate rows

In [13]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
data.duplicated().sum()

np.int64(6240)

In [15]:
data.drop_duplicates(inplace = True)

In [16]:
data.shape

(38638, 2)

#### Randomly shuffle the dataset

In [17]:
data = data.sample(frac = 1)

In [18]:
data.head()

,text,class
19883,LONDON (Reuters) - British police said they ha...,1
545,WASHINGTON (Reuters) - U.S. Attorney General J...,1
1650,NEW YORK (Reuters) - President Donald Trump sa...,1
10284,JK Rowling and the left went on a twitter tira...,0
20744,"FORT LAUDERDALE, Fla. (Reuters) - Hurricane Ha...",1


In [19]:
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [20]:
data.columns

Index(['text', 'class'], dtype='object')

In [21]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W", " ", text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\aarad\AppData\Local\Temp\ipykernel_36664\2198927988.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\aarad\AppData\Local\Temp\ipykernel_36664\2198927988.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\aarad\AppData\Local\Temp\ipykernel_36664\2198927988.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [22]:
data['text'] = data['text'].apply(wordopt)

In [23]:
x = data['text']
y = data['class']

#### Split the training and testing data

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

##### Using Logistic Regression

In [26]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [27]:
pred_lr = LR.predict(xv_test)
accuracy_score(y_test, pred_lr)

0.9842650103519669

In [28]:
LR.score(xv_test, y_test)

0.9842650103519669

In [29]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4359
           1       0.98      0.99      0.99      5301

    accuracy                           0.98      9660
   macro avg       0.98      0.98      0.98      9660
weighted avg       0.98      0.98      0.98      9660



##### Using Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [31]:
pred_dt = DT.predict(xv_test)
accuracy_score(y_test, pred_dt)

0.9946169772256729

In [32]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4359
           1       0.99      1.00      1.00      5301

    accuracy                           0.99      9660
   macro avg       0.99      0.99      0.99      9660
weighted avg       0.99      0.99      0.99      9660



##### Using Gradient Boosting Classifier

In [33]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state = 0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [34]:
pred_gb = GB.predict(xv_test)
accuracy_score(y_test, pred_gb)

0.9954451345755694

In [35]:
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4359
           1       0.99      1.00      1.00      5301

    accuracy                           1.00      9660
   macro avg       1.00      1.00      1.00      9660
weighted avg       1.00      1.00      1.00      9660



##### Using Random Forest

In [36]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state = 0)
RF.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [37]:
pred_rf = RF.predict(xv_test)
accuracy_score(y_test, pred_rf)

0.9853002070393375

In [38]:
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      4359
           1       0.98      0.99      0.99      5301

    accuracy                           0.99      9660
   macro avg       0.99      0.98      0.99      9660
weighted avg       0.99      0.99      0.99      9660



#### Model Testing

In [39]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not a Fake News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_xv_test = vectorization.transform(new_def_test["text"])
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB = GB.predict(new_xv_test)
    pred_RF = RF.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGB Prediction: {} \nRF Prediction: {}".format(output_lable(pred_LR[0]), output_lable(pred_DT[0]), output_lable(pred_GB[0]), output_lable(pred_RF[0])))


In [40]:
news = str(input("Enter the news article: "))
manual_testing(news)



LR Prediction: Not a Fake News 
DT Prediction: Not a Fake News 
GB Prediction: Not a Fake News 
RF Prediction: Not a Fake News


## -------------End--------------------------